# A guide to creating a Kaltura Integration

In [ ]:
from KalturaClient import *
from KalturaClient.Plugins.Core import *
import hashlib
import json
import os

## Step 1: Create app Token (to be done by admin of Kaltura)

In [ ]:
def generateAppToken(adminSecret, adminUserId, partnerId, expiryDuration, sessionUserId, sessionPrivileges):
    
    # generate admin ks
    config = KalturaConfiguration(partnerId)
    config.serviceUrl = "https://www.kaltura.com/"
    client = KalturaClient(config)
    ks = client.session.start(
                adminSecret,
                adminUserId,
                KalturaSessionType.USER,
                partnerId)
    
    client.setKs(ks)
    # create the app token
    appToken = KalturaAppToken()
    appToken.hashType = KalturaAppTokenHashType.SHA256
    appToken.sessionPrivileges = sessionPrivileges
    appToken.sessionType = KalturaSessionType.USER
    appToken.sessionUserId = sessionUserId
    appToken.sessionDuration = expiryDuration

    result = client.appToken.add(appToken);
    res = {"tokenId": result.id, "appToken": result.token}    
    return res

### Fill in the following, and uncomment

In [ ]:
# admin_secret = '' # Available on the kaltura management console
# admin_user_id = '' # Mail id of the kaltura admin
# partner_id = # An integer number, available on the kaltura management console
# expiry_duration = # An integer, duration in seconds for the token to be valid for
# session_user_id = # Kaltura userId to which the appToken will be granted
# session_privileges = # A string containing the privileges to be granted to this token

### Copy the 'tokenId' and 'appToken' generated above and give to whom this token is generated for

In [ ]:
appToken = generateAppToken(admin_secret, admin_user_id, partner_id, expiry_duration, session_user_id, session_privileges)
print(appToken)

## Step 2: Using the appToken generate a Kaltura Client

In [ ]:
def getClient(partnerId, tokenId, appToken):
    config = KalturaConfiguration(partnerId)
    config.serviceUrl = "https://www.kaltura.com/"
    client = KalturaClient(config)
    # generate a widget session in order to use the app token
    widgetId = "_"+str(partnerId)
    expiry = 864000
    result = client.session.startWidgetSession(widgetId, expiry);
    client.setKs(result.ks)    
    
    # generate token hash from ks + appToken
    tokenHash = hashlib.sha256(result.ks.encode('ascii')+appToken.encode('ascii')).hexdigest()    
    # start an app token session
    result = client.appToken.startSession(tokenId, tokenHash, '', '', expiry);
    client.setKs(result.ks)    
    return client

### Uncomment and fill the following code block to generate KalturaClient

In [ ]:
# partner_id = # An integer number, available on the kaltura management console
# tokenId = # Kaltura userId to which the appToken will be granted
# appToken = # A string containing the privileges to be granted to this token

### Use the client object for all further api requests

In [ ]:
client = getClient(partner_id, tokenId, appToken)

### Utilize the functions below to get channels, playlists and media

In [ ]:
def getMediaInfo(mediaId):
    mediaEntry = client.media.get(mediaId, -1)
    media = {'id': mediaEntry.id, 
                'downloadUrl': mediaEntry.downloadUrl, 
                'name': mediaEntry.name,
                'description': mediaEntry.description,
                'createdAt': mediaEntry.createdAt
             }
    return media

def getKalturaPlaylist(kalturaPlaylistId):
    playlist = client.playlist.get(kalturaPlaylistId, -1)
    mediaIds = playlist.getPlaylistContent().split(',')
    return mediaIds

def getKalturaAllChannelIds():
    channels = client.category.list()
    channelIds = [x.id for x in channels.objects]
    return channelIds

def getKalturaChannel(channelId):
    return client.category.get(channelId)
    
def getKalturaChannelEntries(channelId):    
    a = KalturaCategoryEntryFilter()
    a.categoryIdEqual = channelId
    b = KalturaFilterPager()    
    entries = client.categoryEntry.list(a, b)    
    res = []
    for entry in entries.objects:
        res.append(getMediaInfo(entry.entryId))    
    return json.dumps(res)